In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
RT_list = zeros(0,1);
% % roomount = 1;

room_size=[5,4,2.75];
F_abs = [125,250,500,1000,2000,4000];

scale_list = 0.1:0.0001:10;
scale_num = length(scale_list);

for scale_i = 1:scale_num
    A_wall = [0.03 0.15 0.50 0.80 0.85 0.80
              0.03 0.15 0.50 0.80 0.85 0.80
              0.03 0.15 0.50 0.80 0.85 0.08
              0.03 0.15 0.50 0.80 0.85 0.80];

    A_floor = [0.15 0.11 0.10 0.07 0.06 0.07];

    A_roof =  [0.01 0.01 0.01 0.02 0.02 0.02];

    A_all = [A_wall*scale_list(scale_i); A_floor; A_roof]';

    humidity = 50;
    m_air = 5.5E-4*(50/humidity)*(F_abs/1000).^(1.7);
    c= 343;

    RT60_estimator = 'Norris_Eyring';
    [RT60_Air MFP_Air]= reverberation_time(c,room_size,A_all,F_abs,m_air,RT60_estimator);
    RT_list(end+1) = RT60_Air(4);
end

plot(scale_list,RT_list,'linewidth',2);
hold on;
plot([1,1],[0,2],'r');
xlabel('scale'); ylabel('RT(1 kHz)');
saveas(gcf,'RT_curve','bmp');
% close

tar_RT = 1.48:0.08:2.12;
tar_RT = [0.2,0.45,0.7];
tar_RT_len = length(tar_RT);
tar_A = zeros(tar_RT_len,1);
for tar_RT_i = 1:tar_RT_len
    [~,index] = min((tar_RT(tar_RT_i)-RT_list).^2);
    tar_A(tar_RT_i) = scale_list(index);
end
%stem(tar_RT,tar_A);
disp(tar_A');

In [ ]:
function [RT60, MFP]= reverberation_time(c,room_size,A,F_abs,m_air,estimator)

Lx=room_size(1);
Ly=room_size(2);
Lz=room_size(3);
V_room = Lx.*Ly.*Lz; % Volume of room m^3
Sxz=Lx.*Lz;
Syz=Ly.*Lz;
Sxy=Lx.*Ly;
S =2.*(Sxz + Syz + Sxy); % Total area of shoebox room surfaces
Se = Syz.*(A(:,1) + A(:,2)) + Sxz.*(A(:,3) + A(:,4)) + Sxy.*(A(:,5) + A(:,6)); %Effective absorbing area of room surfaces at each frequency
a_bar = Se./S; % Mean absorption of each room surface
m = mean(m_air); % Mean absorption of air averaged across frequency.
MFP = 4*V_room/S; % Mean Free Path (Average distance between succesive reflections) (Ref A4)

% Reverberation time estimate 
if abs(1-a_bar) < eps, % Detect anechoic case and force RT60 all zeros.
    RT60 = zeros(size(F_abs));
else % Select an estimation equation
    switch estimator
        case {'Sabine'}
            RT60 = (55.25/c)*V_room./Se; % Sabine equation
        case {'SabineAir'}
            RT60 = (55.25/c)*V_room./(4*m_air'*V_room+Se); % Sabine equation (SI units) adjusted for air
        case {'SabineAirHiAbs'}
            RT60 = (55.25/c)*V_room./(4*m_air'*V_room+Se.*(1+a_bar/2)); % Sabine equation (SI units) adjusted for air and high absorption
        case {'Norris_Eyring'}
            RT60 = (55.25/c)*V_room./(4*m_air'*V_room-S*log(1-a_bar+eps)); % Norris-Eyring estimate adjusted for air absorption
    end;
end;
%-------------------------- End of reverberation_time.m ---------------------------------------------


In [ ]:
def RT60_estimator(c,room_size,A,freqs,estimator):
    """estimator RT60 based on room size and surface materials(sound absorption coefficients)
    c: sound speed,343
    room_size: [length,width,heigth]
    A: sound aborption coefficents,[freq_chann,6(surfaces)]
    freqs: frequency on which RT60 is estimated
    estimator: estimation method, {"Sabine","SabineAir","SabineAirHiAbs","Norris_Eyring"}
    """
    Lx,Ly,Lz = room_size
    V_room = Lx*Ly*Lz # vlolume of room
    # area of each surface
    Sxy = Lx*Ly
    Sxz = Lx*Ly
    Syz = Ly*Lz
    S = 2*(Lxy+Lxz+Lyz) # total area
    Se = Syz*(A[:,0]+A[:,1]) + Sxz*(A[:,2]+A[:,3]) + Sxy*(A[:,4]+A[:,5])
    A_mean = Se/S # mean absorption coefficients
    
    # absorption coeffeicents of air
    humidity = 50;
    A_air = 5.5E-4*(50/humidity)*(F_abs/1000).^(1.7);
    A_air_bar = np.mean(A_air)
    
    MFP = 4*V_room/S # Mean Free Path(avarage distance between successive reflections)
    
    if np.abs(1-A_air_bar) < np.1e-20:
        RT60 = np.zeros_like(F_abs)
    else:
        if RT60_estimator == 'Sabine':
            RT60 = (55.25/c)*V_room/Se
        elif RT60_estimator == 'SabineAir':
            RT60 = (55.25/c)*V_room/()4*A_air